In [16]:
%%time
import pandas as pd
import pyarrow as pa
import polars as pl
from pyarrow import parquet
from pyarrow import csv

CPU times: total: 0 ns
Wall time: 0 ns


In [94]:
FILE_NAME="A"
FILE_PARQUET="stock_market_data/forbes2000/csv/"+ FILE_NAME + ".parquet"
FILE_CSV = "stock_market_data/forbes2000/csv/"+ FILE_NAME +".csv"
OUTPUT_FILE="consolidated"

In [25]:
%%time
import glob
ALL_FILES_PARQUET = []
for file in glob.glob("stock_market_data/*/csv/*.parquet", recursive = True):
    ALL_FILES_PARQUET.append(file)
display(len(ALL_FILES_PARQUET))

ALL_FILES_CSV = []
for file in glob.glob("stock_market_data/*/csv/*.csv", recursive = True):
    ALL_FILES_CSV.append(file)
display(len(ALL_FILES_CSV))

4193

4193

CPU times: total: 0 ns
Wall time: 52.3 ms


# Read Parquet and simply calculation

In [95]:
FILE_COUNT=500

## Polars

In [89]:
%%time
df1_polars = pl.read_parquet(ALL_FILES_PARQUET[0])
for file in ALL_FILES_PARQUET[1:FILE_COUNT]:
    df1 = pl.read_parquet(FILE_PARQUET)
    df1_polars=pl.concat([df1,df1_polars])
    #print(len(df1_polars))

CPU times: total: 3.22 s
Wall time: 12.8 s


In [90]:
%%time
df1_polars.with_columns(Range = pl.col("High") + pl.col("Low"))

CPU times: total: 0 ns
Wall time: 17.1 ms


Date,Low,Open,Volume,High,Close,Adjusted Close,Range
str,f64,f64,i64,f64,f64,f64,f64
"""18-11-1999""",28.612303,32.546494,62546380,35.765381,31.473534,27.066582,64.377684
"""19-11-1999""",28.478184,30.713518,15234146,30.758226,28.880545,24.836662,59.23641
"""22-11-1999""",28.657009,29.551144,6577870,31.473534,31.473534,27.066582,60.130543
"""23-11-1999""",28.612303,30.400572,5975611,31.205294,28.612303,24.60598,59.817596
"""24-11-1999""",28.612303,28.701717,4843231,29.998213,29.372318,25.259573,58.610516
"""26-11-1999""",29.148785,29.238197,1729466,29.685265,29.461731,25.336472,58.834049
"""29-11-1999""",29.014664,29.327612,4074751,30.355865,30.132332,25.913176,59.370529
"""30-11-1999""",29.282904,30.042917,4310034,30.713518,30.177038,25.95162,59.996422
"""01-12-1999""",29.953505,30.177038,2957329,31.071173,30.713518,26.412981,61.024677


In [69]:
df1_polars.dtypes

[Utf8, Float64, Float64, Int64, Float64, Float64, Float64]

In [96]:
# <class 'polars.dataframe.frame.DataFrame'>
print(type(df1_polars))
# 'polars.dataframe.frame.DataFrame' is backed by arrow im-memory representation rather than numpy

<class 'polars.dataframe.frame.DataFrame'>


## Pyarrow

In [100]:
%%time
pydict=dict()
df1_pyarrow = pa.parquet.read_table(ALL_FILES_PARQUET[0])
for file in ALL_FILES_PARQUET[1:FILE_COUNT]:
    df1= pa.parquet.read_table(file)
    df1_pyarrow = pa.concat_tables([df1,df1_pyarrow])
    #print(len(df1_pyarrow))

ArrowInvalid: Schema at index 1 was different: 
Date: string
Low: double
Open: double
Volume: double
High: double
Close: double
Adjusted Close: double
vs
Date: string
Low: double
Open: double
Volume: int64
High: double
Close: double
Adjusted Close: double

In [101]:
df1_pyarrow.info()

AttributeError: 'pyarrow.lib.Table' object has no attribute 'info'

In [7]:
# <class 'pyarrow.lib.Table'>
print(type(df1))

<class 'pyarrow.lib.Table'>


## Pandas

In [92]:
%%time
df1_pandas = pd.DataFrame()
for file in ALL_FILES_PARQUET[:FILE_COUNT]:
    df1=pd.read_parquet(file)
    df1_pandas=pd.concat([df1,df1_pandas])

CPU times: total: 14.4 s
Wall time: 38.2 s


In [93]:
%%time
df2=df1_pandas["range"] = df1_pandas.High - df1_pandas.Low

CPU times: total: 0 ns
Wall time: 27.8 ms


In [76]:
df2.info()

<class 'pandas.core.series.Series'>
Index: 54379 entries, 0 to 5803
Series name: None
Non-Null Count  Dtype  
--------------  -----  
54379 non-null  float64
dtypes: float64(1)
memory usage: 849.7 KB


In [ ]:
df1_pandas.info()

In [103]:
#<class 'pandas.core.frame.DataFrame'>
print(type(df1_pandas))

<class 'pandas.core.frame.DataFrame'>


# Read CSV

## Polars

In [10]:
%%time
for file in ALL_FILES_CSV[:FILE_COUNT]:
    df1 = pl.read_csv(file)

CPU times: total: 281 ms
Wall time: 1.29 s


## Pyarrow

In [11]:
%%time
for file in ALL_FILES_CSV[:FILE_COUNT]:
    df1=pa.csv.read_csv(file)

CPU times: total: 406 ms
Wall time: 2.53 s


## Pandas

In [12]:
%%time
for file in ALL_FILES_CSV[:FILE_COUNT]:
    df1=pd.read_csv(file)

CPU times: total: 3.62 s
Wall time: 8.48 s


# Experiments

In [14]:
%%time
import polars as pl 
pl.read_parquet("test_parquet/*.parquet")

CPU times: total: 0 ns
Wall time: 6.96 ms


Date,Low,Open,Volume,High,Close,Adjusted Close
str,f64,f64,i64,f64,f64,f64
"""18-11-1999""",28.612303,32.546494,62546380,35.765381,31.473534,27.066582
"""19-11-1999""",28.478184,30.713518,15234146,30.758226,28.880545,24.836662
"""22-11-1999""",28.657009,29.551144,6577870,31.473534,31.473534,27.066582
"""23-11-1999""",28.612303,30.400572,5975611,31.205294,28.612303,24.60598
"""24-11-1999""",28.612303,28.701717,4843231,29.998213,29.372318,25.259573
"""26-11-1999""",29.148785,29.238197,1729466,29.685265,29.461731,25.336472
"""29-11-1999""",29.014664,29.327612,4074751,30.355865,30.132332,25.913176
"""30-11-1999""",29.282904,30.042917,4310034,30.713518,30.177038,25.95162
"""01-12-1999""",29.953505,30.177038,2957329,31.071173,30.713518,26.412981


In [71]:
%%time
df_consolidate_polars = pl.read_parquet("10_file_folder/*.parquet")

CPU times: total: 31.2 ms
Wall time: 16.5 ms


# Analyse data

## Pandas

# Concat all parquet into one using Pandas (25 minutes)

In [31]:
%%time
df1_all = pd.DataFrame()
for file in ALL_FILES_PARQUET[:FILE_COUNT]:
    df1=pd.read_parquet(file)
    df1_all=pd.concat([df1,df1_all])
    # print(len(df1_all))

df1_all.to_parquet(OUTPUT_FILE+"_pandas.parquet")

CPU times: total: 8min 57s
Wall time: 24min 58s


In [35]:
df1_all.to_parquet(OUTPUT_FILE+"_pandas.parquet")

In [34]:
df1_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24604971 entries, 0 to 5803
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   Date            object 
 1   Low             float64
 2   Open            float64
 3   Volume          float64
 4   High            float64
 5   Close           float64
 6   Adjusted Close  float64
dtypes: float64(6), object(1)
memory usage: 1.5+ GB


## Copy files into a smaller folder

In [70]:
import shutil
FILE_COUNT=10
DEST = r"10_file_folder"
for file in ALL_FILES_PARQUET[:FILE_COUNT]:
    shutil.copy(file, DEST)